# Imports

In [ ]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from analysis.dim_reduction import Data
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity

# Data

In [ ]:
# путь к папке с обработанными minian данными 
path_to_data = 'demo_movies'

# рассматриваемые сесси
# название
# path - путь к файлам
# label - группа
# fps - количество кадров в секунду
dates = {
    'mouse_old_1': {'path': 'mouse_old/25_october_2021',
                    'label': 'mouse old',
                    'fps': 20},
    'mouse_old_2': {'path': 'mouse_old/11_november_2021',
                    'label': 'mouse old',
                    'fps': 20},
    'mouse_old_3': {'path': 'mouse_old/15_november_2021',
                    'label': 'mouse old',
                    'fps': 20},
    
    'mouse1_1': {'path': 'mouse1/12_october_2022',
                 'label': 'mouse 1',
                 'fps': 15},
    'mouse1_2': {'path': 'mouse1/13_october_2022',
                 'label': 'mouse 1',
                 'fps': 15},
    'mouse1_3': {'path': 'mouse1/14_october_2022',
                 'label': 'mouse 1',
                 'fps': 15},
    
    'mouse2_1':{'path': 'mouse2/12_october_2022',
                 'label': 'mouse 2',
                 'fps': 15},
    'mouse2_2': {'path': 'mouse2/13_october_2022',
                 'label': 'mouse 2',
                 'fps': 15},
    'mouse2_3': {'path': 'mouse2/14_october_2022',
                 'label': 'mouse 2',
                 'fps': 15},
}

In [ ]:
%%time
# подгрузка данных
data_class = Data(path_to_data, dates, verbose=True)

In [ ]:
%%time
# сбор статистик
# операция занимает продолтительное время
df = data_class.get_data()

In [ ]:
# удалние сильноскоррелированных признаков
df = Data.drop_strong_corr(df)

In [ ]:
# снижение размерности
data, pca = data_class.data_reduction(df)

In [ ]:
# визуализация результатов
Data.show_result(data)

In [ ]:
# табличное представление результатов
data['date'] = df.index
result = data.copy()
result

In [ ]:
# сохраниение результатов
result.to_csv('reduced_data.csv')

# PCA

In [ ]:
# веса признаков из PCA
feature_importance = pd.DataFrame(pca.components_, columns=df.columns)
feature_importance

In [ ]:
# центры масс по каждой категории записей
centers = result.pivot_table(values=['x', 'y'], index='label', aggfunc='mean')
centers

In [ ]:
# подсчет косинусного сходства
features = {'value': [],
             'label': [],
             'feature': [],
             'rating': []
            }
top_n = 8
for i, row in centers.iterrows():
    top = feature_importance.apply(lambda col: cosine_similarity([row], [col])[0,0] * (col ** 2).sum() ** (1 / 2))
    top = top[top.abs().sort_values(ascending=False).head(top_n).index]
    features['value'] += top.tolist()
    features['feature'] += top.index.tolist()
    features['label'] += [i] * (top_n)
    features['rating'] += list(np.arange(top_n))
    
features = pd.DataFrame(features)
features = features.pivot_table(values=['value', 'rating'], index=['label', 'feature'])
features = features.sort_values(by=['label', 'rating'])
features

In [ ]:
# визуализация наиболее выжных призанков по косинусному сходству
feature_df = features.reset_index()

fig, axs = plt.subplots(3, 1, figsize=(12, 15))
fig.subplots_adjust(hspace=.3)


for ax, label in zip(axs, feature_df['label'].unique()):
    
    sns.barplot(data = feature_df[feature_df['label'] == label], y='feature', x='value', ax=ax)
    ax.set_title(label, fontsize=18)
    
    ax.set_xlabel('value', fontsize=14)
    ax.set_ylabel('feature', fontsize=14)
    ax.tick_params(axis='both', labelsize=14)
    
plt.show()